In [1]:
!rm -rf Eunomia
!git clone https://github.com/ouaguenouni/Eunomia
!pip install -r Eunomia/requirements.txt

Cloning into 'Eunomia'...
remote: Enumerating objects: 454, done.
remote: Counting objects: 100% (454/454), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 454 (delta 347), reused 403 (delta 296), pack-reused 0
Receiving objects: 100% (454/454), 2.91 MiB | 2.17 MiB/s, done.
Resolving deltas: 100% (347/347), done.


In [2]:
import itertools
import random
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from Eunomia.preferences import *
from Eunomia.additive_functions import *
from Eunomia.alternatives import *
from Eunomia.sampling import *
from Eunomia.mcmc import *
#from Eunomia.degree import *
from Eunomia.experiments import *
from Eunomia.ssb import * 
from Eunomia.graph_utils import * 

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm, trange
pyro.set_rng_seed(1)
import itertools
import random
import networkx as nx

def generate_random_directed_graph(n, m, p_edge):
    # Generate the set F = {0, 1, ..., n}
    F = set(range(n + 1))

    # Generate all subsets of F
    all_subsets = list(itertools.chain.from_iterable(itertools.combinations(F, r) for r in range(len(F)+1)))

    # Sample m subsets from F to be the vertices
    vertices = random.sample(all_subsets, m)

    # Create a directed graph with these vertices
    G = nx.DiGraph()
    G.add_nodes_from(vertices)

    # Add directed edges between these vertices with probability p_edge
    for i in vertices:
        for j in vertices:
            if i != j:
                if random.random() < p_edge:
                    # Add edge (i, j) only if (j, i) doesn't exist
                    if not G.has_edge(j, i):
                        G.add_edge(i, j)

    return G



# Now G is a randomly sampled directed graph


In [3]:
# Example usage
n = 3 # Maximum number in set F
k = 2
p_edge = 0.5  # Probability of creating an edge
theta = [tuple(i) for i in generate_additive_theta(n, k)]

In [4]:
graphs = {}
num_graphs = 2000
for _ in trange(num_graphs):
    G = generate_random_directed_graph(n, 2**n, p_edge)
    circuits = list(nx.simple_cycles(G))
    graphs[len(circuits)] = graphs.get(len(circuits), []) + [G.edges()]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [5]:
data = {
    "n_cycles":[],
    "additive_acc":[],
    "nt_acc":[]
}
n_epochs = 200
lr=5e-3
m = 10
for k in tqdm(sorted(graphs.keys())):
    L = []
    nb = 0
    if len(graphs[k]) < m:
        continue
    for s in tqdm(graphs[k][:m]):
        model_map = SSBModel(theta)
        acc1 = fit_w_mle(model_map, s, num_epochs=n_epochs,  lr=5e-2)
        acc2 = fit_M_mle(model_map, s, num_epochs=n_epochs, lr=lr)
        acc = acc1 + acc2
        #plt.plot(np.arange(len(acc)), acc)
        #plt.show()
        data["n_cycles"].append(k)
        data["additive_acc"].append(max(acc1))
        data["nt_acc"].append(max(acc2))
        L.append(max(acc2) - max(acc1))
    print(f"{k} - {np.array(L).mean()}")

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

NameError: name 'fit_w_mle' is not defined